# Introduction to Practical Artificial Intelligence. Final Exam

Tishkin Pavel p.tishkin@innopolis.university

# Problem 3. Minimax

[75%] Implement a minimax tree for a simplified version of Halma. You are given 3x3 field and 3 pieces each:

You need to move pieces in the “Home” of your opponent (colored). Who finished first — wins. If you cannot finish in 16 moves — draw.

Moves are done into a spare neighbour cell (8-connectivity), OR by jumping over the piece (no matter, your or opponent’s). E.g. debuts like these are allowed:

 - A2-B2 ; C2-A2
 - A1-B2 ; C3-A1
 - B1-C1 ; C3-B2

[25%] Visualize the game which leads to WIN. OR implement a bot to play with.

In [1]:
import copy
import itertools
import numpy as np
import math

# Solution based on HW and lab on minimax bot of tic-tac-toe

class Field(object):
    # state is a 3x3 char matrix
    # Game pieces are going to be represented by:
    # 'x' - player 1's piece
    # 'o' - player 2's piece
    state = []
    turn = 0
    children = []
    minimax_score = None
    x_points = None
    o_points = None
    
    def __init__(self, state, turn):
        self.state = state
        self.turn = turn
        self.x_points = np.argwhere(np.array(state) == 'x')
        self.o_points = np.argwhere(np.array(state) == 'o')
        
    def __str__(self):
        '''
        this is a field representation code
        '''
        
        res = "  | A | B | C |\n-------------------\n"
        for i, line in enumerate(self.state):
            res += "{} | {} | {} | {} |\n".format(i+1, *line)
            res += "-------------------\n"
        return res
    
    def __repr__(self):
        '''
        this is a short form to represent a field
        '''
        return "".join(itertools.chain(*self.state)) + f'{self.turn}'
    
    def get(self, tpl):
        # Done
        '''
        returns a characted in a given position
        '''
        return self.state[tpl[0]][tpl[1]]
    
    def make_a_move(self, tpl):
        '''
        Makes a move and returns a new field
        '''
        who = self.is_move_of()
        state = copy.deepcopy(self.state)
        state[tpl[0]][tpl[1]] = ' '
        state[tpl[2]][tpl[3]] = who
        return Field(state, self.turn + 1)
    
    def is_move_of(self):
        '''
        returns a piece if move can be done and a None if game is over
        '''
        if self.is_win_of() is not None:
            return None
        
        return 'x' if self.turn % 2 == 0 else 'o'
    
    def is_win_of(self):
        '''
        Checks who is winning
        '''
        if (self.state[1][2] == self.state[2][1] == self.state[2][2] == 'x'):
            return 'x'
        if (self.state[0][0] == self.state[1][0] == self.state[0][1] == 'o'):
            return 'o'
        # Maximal depth
        if self.turn == 16:
            return ' '
        # if game is not over -> return None
        return None
    
    def get_next_move(self):
        '''
        Gets the best move for the bot
        '''
        max_score = 0
        best_ch = []
        for ch in self.children:
            if (ch.minimax_score > max_score):
                # in case of vizhu maat v 4 hoda, a ne v 1
                if (ch.is_win_of() == 'x'):
                    return ch
                max_score = ch.minimax_score
                best_ch = [ch]
            if (ch.minimax_score == max_score):
                best_ch.append(ch)
        return np.random.choice(best_ch)

In [2]:
def is_possible_move(field, side, start, end, debug=False):
    '''
    Checks if move is possible
    '''
    # Checking if every element in start and end does not exceed boundaries
    # Of the board
    if not all([m <= 2 and m >= 0 for m in np.append(start, end)]):
        return False
    # Field must be empty
    if field.state[end[0]][end[1]] != ' ':
        return False
    # Checking for the move through two cells
    # Idea - //2 of abs value of dx gives 1 if moved for 2 cells, 0 if moved for 1 cell
    # If we add such value to teh start with the correct sign, we will either check the middle point between
    # Start and end, in case of move=2 cells; and check the starting cell (which always returns true) in case of move=1 cell
    dx = (end[0] - start[0])
    dy = (end[1] - start[1])
    middle_x = int(math.copysign(abs(dx) // 2, dx)) + start[0]
    middle_y = int(math.copysign(abs(dy) // 2, dy)) + start[1]
    # If change was only for one cell, dx=dy=0, cell[start]==cell[start+0]
    # If change was in two cells, dx and dy will give the coordinates of the between cell
    if (field.state[middle_x][middle_y] == ' '):
        return False
    return True

def get_move_tuple(notation, field):
    '''
    Converts notation like 'c1c2' into a tuple (2, 0, 2, 1)
    '''
    notation = notation.lower()
    if len(notation) != 4:
        return None
    if (notation[0] not in 'abc') or (notation[1] not in '123') or (notation[2] not in 'abc') or (notation[3] not in '123'):
        return None
    translation = {'a': 0, 'b': 1, 'c': 2,}
    start, end = [int(notation[1]) - 1, translation[notation[0]]], [int(notation[3]) - 1, translation[notation[2]]]
    if is_possible_move(field, 'o', start, end, True):
        return (start[0], start[1], end[0], end[1])
    return None

In [3]:
import copy
    
def generate_children_for_the_field(field):
    '''
    Generates children for the field
    '''
    result = []
    # Singular moves
    movements = [np.array([1, 0]), np.array([-1, 0]), np.array([0, 1]), np.array([0, -1]), np.array([1, 1]), np.array([-1, 1]), np.array([1, -1]), np.array([-1, -1])]
    # Double moves
    movements.extend((np.array(movements) * 2))
    if field.is_win_of() is None:
        who_moves = field.is_move_of()
        points = field.x_points if who_moves == 'x' else field.o_points
        for p in points:
            for m in movements:
                new_p = p+m
                if is_possible_move(field, who_moves, p, new_p):
                    state = copy.deepcopy(field.state)
                    state[p[0]][p[1]] = ' '
                    state[new_p[0]][new_p[1]] = who_moves
                    child = Field(state, field.turn + 1)
                    result.append(child)
    field.children = result
    return result

In [4]:
def update_minimax(field):
    '''
    Updates minimax
    If the state is not terminal but there it is max depth
    assigns a 0
    Win - 1
    lose - -1
    Draw - 0.5
    '''
    field.minimax_score = 0
    win = field.is_win_of()
    if win != None:
        if win == 'x':
            field.minimax_score = 1
        if win == 'o':
            field.minimax_score = -1
        if win == ' ':
            # Fixes depth
            field.minimax_score = 0.5
    else:
        side = field.is_move_of()
        scores = []
        for ch in field.children:
            update_minimax(ch)
            scores.append(ch.minimax_score)
        if not scores:
            field.minimax_score = 0
        else:
            if side == 'x':
                field.minimax_score = max(scores)
            else:
                field.minimax_score = min(scores)
    return field.minimax_score

In [5]:
# generate a tree
# store it as a map {str -> Field}
def generate_tree(initial, depth=5, debug=False):
    '''
    Generates a tree to some depth
    '''
    states = {initial.__repr__(): initial}
    queue = [(initial, 0)]
    while queue:
        curr = queue.pop()
        children = generate_children_for_the_field(curr[0])
        for child in children:
            states[child.__repr__()] = child
            if depth > curr[1]:
                queue.append((child, curr[1]+1))
    update_minimax(initial)
    if debug:
        print('Total states:', len(states))
        ## BTW, is this ok that some nodes in a tree have the same repr?
        print("Root score: ", initial.minimax_score)
    
    return states

In [6]:
from IPython.display import clear_output

state0 = [['x', 'x', ' '], ['x', ' ', 'o'], [' ', 'o', 'o']]
initial = Field(state0, 0)

field = initial
while field.is_win_of() is None:
    states = generate_tree(field)
    # make a bot move
    field = field.get_next_move()
    # show it
    print(field)
    # if bot wins
    if field.is_win_of() is not None:
        break
    # ask for a move
    m = input("Your move:")
    tpl = get_move_tuple(m, field)
    while tpl is None:
        print("Provide something like `b3b2` of an empty field")
        m = input("Your move:")
        tpl = get_move_tuple(m, field)
    # first build a representation, then retrieve a field from the tree
    field = states[field.make_a_move(tpl).__repr__()]

vic = field.is_win_of()
    
if vic == ' ':
    print('Draw')
else:
    print(field.is_win_of(), "wins")

  | A | B | C |
-------------------
1 | x | x |   |
-------------------
2 |   | x | o |
-------------------
3 |   | o | o |
-------------------



Your move: c2c1


  | A | B | C |
-------------------
1 | x |   | o |
-------------------
2 |   | x | x |
-------------------
3 |   | o | o |
-------------------



Your move: b3b1


  | A | B | C |
-------------------
1 |   | o | o |
-------------------
2 | x | x | x |
-------------------
3 |   |   | o |
-------------------



Your move: c3a1


  | A | B | C |
-------------------
1 | o | o | o |
-------------------
2 | x |   | x |
-------------------
3 |   |   | x |
-------------------



Your move: c1b2


  | A | B | C |
-------------------
1 | o | o |   |
-------------------
2 |   | o | x |
-------------------
3 |   | x | x |
-------------------

x wins
